In [3]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from FAB2 import *

learning_rate = 0.001  
gamma = 1
buffer_limit = 10000
batch_size = 16

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(64,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 9)
        else:
            return out.argmax().item()
        
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        return out.argmax().item(),out
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 10
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        #여기는 sample_action 구간
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
            
        #학습구간    
        if memory.size()>2000:
            train(q, q_target, memory, optimizer)
        
        #결과 및 파라미터 저장    
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            params = q.state_dict()
            param_name = str(n_epi)+"q.pt"
            #print(param_name)
            torch.save(params, param_name)
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        
        #여기는 select_action 구간
        s = env.reset()
        done = False
        score = 0.0
        params = q.state_dict()
        torch.save(params, "q.pt" )
        while not done:
            a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
            #print(a_list)
            #print(a)
            s_prime, r, done = env.step(a)
            #print(r)
            s = s_prime
            score += r
            if done:
                break
        Flow_time, machine_utill ,util, makespan = env.performance_measure()
        print("--------------------------------------------------")
        print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
        print("n_epi: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),0))
        
        
        
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
    
    
    s = env.reset()
    done = False
    score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        #print(a_list)
        #print(a)
        s_prime, r, done = env.step(a)
        #print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    #env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
for i in range(1):
    Flow_time, machine_util, util, makespan, score =main()
    print("FlowTime:" , Flow_time)
    print("machine_util:" , machine_util)
    print("util:" , util)
    print("makespan:" , makespan)
    print("Score" , score)
    
    
params = torch.load("8q.pt")
q = Qnet()
q.load_state_dict(params)
q.eval()
env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1) 
s = env.reset()
done = False
score = 0.0
epsilon = max(0.01 , 0.08 - 0.02*(20/200))
while not done:
    a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
    #print(a_list)
    #print(a)
    s_prime, r, done = env.step(a)
    #print(r)
    s = s_prime
    score += r
    if done:
        break
Flow_time, machine_util, util, makespan = env.performance_measure()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)    

--------------------------------------------------
flow time: 39877, util : 0.921, makespan : 893
n_epi: 0, score : -5316.0, n_buffer : 340, eps : 0.0%
--------------------------------------------------
flow time: 43704, util : 0.909, makespan : 944
n_episode: 1, score : -5559.0, n_buffer : 680, eps : 8.0%
--------------------------------------------------
flow time: 39877, util : 0.921, makespan : 893
n_epi: 1, score : -5316.0, n_buffer : 680, eps : 0.0%
--------------------------------------------------
flow time: 48845, util : 0.913, makespan : 972
n_episode: 2, score : -5783.0, n_buffer : 1020, eps : 8.0%
--------------------------------------------------
flow time: 39877, util : 0.921, makespan : 893
n_epi: 2, score : -5316.0, n_buffer : 1020, eps : 0.0%
--------------------------------------------------
flow time: 48556, util : 0.907, makespan : 987
n_episode: 3, score : -5823.0, n_buffer : 1360, eps : 8.0%
--------------------------------------------------
flow time: 39877, util

--------------------------------------------------
flow time: 30184, util : 0.905, makespan : 770
n_episode: 27, score : -4579.0, n_buffer : 9520, eps : 7.7%
--------------------------------------------------
flow time: 35328, util : 0.887, makespan : 764
n_epi: 27, score : -4567.0, n_buffer : 9520, eps : 0.0%
--------------------------------------------------
flow time: 34366, util : 0.893, makespan : 782
n_episode: 28, score : -4650.0, n_buffer : 9860, eps : 7.7%
--------------------------------------------------
flow time: 27303, util : 0.891, makespan : 729
n_epi: 28, score : -4334.0, n_buffer : 9860, eps : 0.0%
--------------------------------------------------
flow time: 28100, util : 0.884, makespan : 746
n_episode: 29, score : -4430.0, n_buffer : 10000, eps : 7.7%
--------------------------------------------------
flow time: 26736, util : 0.912, makespan : 755
n_epi: 29, score : -4473.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 

--------------------------------------------------
flow time: 29152, util : 0.897, makespan : 704
n_epi: 53, score : -4129.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 30516, util : 0.881, makespan : 723
n_episode: 54, score : -4289.0, n_buffer : 10000, eps : 7.5%
--------------------------------------------------
flow time: 24755, util : 0.905, makespan : 718
n_epi: 54, score : -4070.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 26784, util : 0.885, makespan : 761
n_episode: 55, score : -4209.0, n_buffer : 10000, eps : 7.4%
--------------------------------------------------
flow time: 24965, util : 0.937, makespan : 819
n_epi: 55, score : -4637.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 26246, util : 0.934, makespan : 817
n_episode: 56, score : -4665.0, n_buffer : 10000, eps : 7.4%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 35989, util : 0.880, makespan : 818
n_episode: 80, score : -4859.0, n_buffer : 10000, eps : 7.2%
--------------------------------------------------
flow time: 24711, util : 0.906, makespan : 738
n_epi: 80, score : -4083.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 25774, util : 0.887, makespan : 702
n_episode: 81, score : -4152.0, n_buffer : 10000, eps : 7.2%
--------------------------------------------------
flow time: 25420, util : 0.939, makespan : 821
n_epi: 81, score : -4641.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 27157, util : 0.921, makespan : 832
n_episode: 82, score : -4777.0, n_buffer : 10000, eps : 7.2%
--------------------------------------------------
flow time: 24710, util : 0.905, makespan : 738
n_epi: 82, score : -4088.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow ti

--------------------------------------------------
flow time: 24646, util : 0.933, makespan : 683
n_epi: 106, score : -4030.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 26243, util : 0.923, makespan : 715
n_episode: 107, score : -4246.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
flow time: 24710, util : 0.905, makespan : 738
n_epi: 107, score : -4088.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 25715, util : 0.889, makespan : 768
n_episode: 108, score : -4165.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
flow time: 32867, util : 0.892, makespan : 752
n_epi: 108, score : -4281.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 32191, util : 0.893, makespan : 773
n_episode: 109, score : -4372.0, n_buffer : 10000, eps : 6.9%
--------------------------------------------------
f

...

--------------------------------------------------
flow time: 35945, util : 0.899, makespan : 758
n_epi: 886, score : -4414.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 35681, util : 0.903, makespan : 738
n_episode: 887, score : -4383.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 23828, util : 0.924, makespan : 725
n_epi: 887, score : -4018.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 24589, util : 0.906, makespan : 745
n_episode: 888, score : -4062.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 24986, util : 0.951, makespan : 826
n_epi: 888, score : -4706.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 24986, util : 0.951, makespan : 826
n_episode: 889, score : -4706.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
f

--------------------------------------------------
flow time: 27140, util : 0.966, makespan : 814
n_epi: 912, score : -4793.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 27440, util : 0.962, makespan : 806
n_episode: 913, score : -4782.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 37694, util : 0.892, makespan : 658
n_epi: 913, score : -3888.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 37451, util : 0.893, makespan : 654
n_episode: 914, score : -3853.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 25148, util : 0.944, makespan : 775
n_epi: 914, score : -4436.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 25192, util : 0.943, makespan : 771
n_episode: 915, score : -4472.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
f

--------------------------------------------------
flow time: 24522, util : 0.946, makespan : 753
n_epi: 938, score : -4299.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 24732, util : 0.941, makespan : 757
n_episode: 939, score : -4380.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 26417, util : 0.946, makespan : 825
n_epi: 939, score : -4883.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 26539, util : 0.945, makespan : 832
n_episode: 940, score : -4865.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 46759, util : 0.867, makespan : 785
n_epi: 940, score : -4652.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 46759, util : 0.867, makespan : 785
n_episode: 941, score : -4652.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
f

--------------------------------------------------
flow time: 29779, util : 0.927, makespan : 836
n_epi: 964, score : -4967.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 29842, util : 0.927, makespan : 818
n_episode: 965, score : -4883.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 41134, util : 0.869, makespan : 835
n_epi: 965, score : -4964.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 42585, util : 0.864, makespan : 851
n_episode: 966, score : -5040.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 37887, util : 0.891, makespan : 672
n_epi: 966, score : -3953.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 38216, util : 0.887, makespan : 680
n_episode: 967, score : -4022.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
f

--------------------------------------------------
flow time: 37493, util : 0.884, makespan : 663
n_epi: 990, score : -3903.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 37776, util : 0.885, makespan : 660
n_episode: 991, score : -3906.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 26880, util : 0.958, makespan : 873
n_epi: 991, score : -4815.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 26880, util : 0.958, makespan : 873
n_episode: 992, score : -4815.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
flow time: 25475, util : 0.920, makespan : 750
n_epi: 992, score : -4432.0, n_buffer : 10000, eps : 0.0%
--------------------------------------------------
flow time: 27422, util : 0.909, makespan : 777
n_episode: 993, score : -4555.0, n_buffer : 10000, eps : 1.0%
--------------------------------------------------
f